# import

In [ ]:
from __future__ import annotations
from pymongo import MongoClient
from dotenv import load_dotenv
from pymongo.errors import ConnectionFailure
import os
from pathlib import Path
from typing import Dict, List, Optional
import re, json, requests


# DB연결
## get_mongo_postgre_db.py

[DB관련]

데이터베이스 연결을 관리하고 생성하는 모듈.

이 모듈은 어플리케이션의 다른 부분에서 필요로 하는 MongoDB와 PostgreSQL
데이터베이스 연결 객체를 생성하는 함수를 제공합니다. 연결 정보는 .env 파일에서
환경 변수를 로드하여 사용합니다.

주요 기능:
- `get_mongodb()`: MongoDB 서버에 연결하고 데이터베이스 객체를 반환합니다.
- `get_postgres()`: PostgreSQL 서버에 연결하고 커넥션 객체를 반환합니다.
- PostgreSQL 드라이버(`psycopg2`)가 없을 경우, 사용자에게 알리는 예외 처리를 포함합니다.

이 스크립트를 직접 실행하면 각 데이터베이스에 대한 연결 테스트를 수행합니다.

In [ ]:
try:
    import psycopg2
    POSTGRES_AVAILABLE = True
except ImportError:
    POSTGRES_AVAILABLE = False
    print("psycopg2가 설치되지 않았습니다. PostgreSQL 기능을 사용할 수 없습니다.")

# mongodb 연결
def get_mongodb():
    load_dotenv()

    USERNAME = os.getenv("MONGO_INITDB_ROOT_USERNAME")
    PASSWORD = os.getenv("MONGO_INITDB_ROOT_PASSWORD")
    HOST = os.getenv("MONGO_HOST")
    PORT = int(os.getenv("MONGO_PORT"))

    url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"

    try:
        client = MongoClient(url)
        client.admin.command('ping')
        print("Successfully connected to MongoDB!")
        db = client['md_db']
        print("✅ MongoDB 연결 성공")
        return db

    except ConnectionFailure as e:
        print(f"MongoDB connection failed: {e}")

# PostgreSQL 연결
def get_postgres():
    load_dotenv()

    password = os.getenv('POSTGRES_PASSWORD')
    if not password:
        raise ValueError("POSTGRES_PASSWORD 환경 변수가 설정되지 않았습니다. .env 파일을 확인하세요.")

    return psycopg2.connect(
        host=os.getenv('PG_HOST', 'localhost'),
        port=int(os.getenv('PG_PORT', 5432)),
        database=os.getenv('PG_DATABASE', 'CoreDB'),
        user=os.getenv('PG_USER', 'promtree'),
        password=password,
        connect_timeout=10
    )

print("=" * 50)
print("DB 연결 테스트")
print("=" * 50)

mongodb = get_mongodb()
postgres = get_postgres()

if mongodb is not None:
    print(f"사용 가능한 컬렉션: {mongodb.list_collection_names()}")

if mongodb is not None:
    cursor = postgres.cursor()
    cursor.execute("SELECT version();")
    version = cursor.fetchone()
    print("✅ nPostgreSQL 연결 성공")
    postgres.close()

## msds_db_create_pg_tables.py

[DB관련]

MSDS 데이터 저장을 위한 PostgreSQL 데이터베이스 스키마 관리 모듈.

이 모듈은 MSDS(물질안전보건자료) 데이터를 저장하기 위한 PostgreSQL 테이블과
인덱스를 생성, 초기화하고, 파싱된 데이터를 데이터베이스에 저장(upsert)하는
함수들을 포함합니다.

주요 기능:
- `init_msds_schema`: 'products', 'ingredients', 'ingredient_synonyms'
  테이블과 관련 인덱스를 모두 생성하여 스키마를 초기화합니다.
- `save_current_parse_to_postgres`: 파싱된 제품 및 성분 정보를 받아
  데이터베이스에 저장합니다.
- 테이블 생성, 인덱스 생성, 데이터 삽입(Insert)/업데이트(Upsert)를 위한
  개별 함수들을 제공합니다.

In [ ]:
def create_products_table(conn):
    """제품(MSDS 문서) 정보 저장 테이블 생성"""
    with conn.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS products (
                id BIGSERIAL PRIMARY KEY,
                file_name TEXT NOT NULL,
                document_id TEXT NOT NULL,
                product_name TEXT NOT NULL,
                company_name TEXT,
                UNIQUE (file_name, product_name)
            );
        """)
    conn.commit()
    print("products 테이블 생성 완료")


def create_ingredients_table(conn):
    """성분 정보 저장 테이블 생성"""
    with conn.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ingredients (
                id BIGSERIAL PRIMARY KEY,
                product_id BIGINT NOT NULL,
                name TEXT,
                cas TEXT,
                ec_number TEXT,
                conc_raw TEXT,
                conc_value NUMERIC,
                conc_min NUMERIC,
                conc_max NUMERIC,
                conc_unit TEXT,
                conc_basis TEXT,
                conc_op_min TEXT,
                conc_op_max TEXT,
                synonym_jsonb JSONB,
                additional_info JSONB,
                CONSTRAINT fk_ingredients_products
                    FOREIGN KEY (product_id) REFERENCES products(id)
                    ON DELETE CASCADE
            );
        """)
    conn.commit()
    print("ingredients 테이블 생성 완료")


def create_ingredient_synonyms_table(conn):
    """성분 동의어 저장 테이블 생성"""
    with conn.cursor() as cursor:
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS ingredient_synonyms (
                id BIGSERIAL PRIMARY KEY,
                ingredient_id BIGINT NOT NULL,
                synonym TEXT NOT NULL,
                CONSTRAINT uq_ing_syn UNIQUE (ingredient_id, synonym),
                CONSTRAINT fk_syn_ingredient
                    FOREIGN KEY (ingredient_id) REFERENCES ingredients(id)
                    ON DELETE CASCADE
            );
        """)
    conn.commit()
    print("ingredient_synonyms 테이블 생성 완료")


def create_indexes(conn):
    """데이터 조회 성능 향상을 위해 주요 컬럼에 인덱스를 생성"""
    with conn.cursor() as cursor:
        # CAS 번호 조회용 인덱스
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_ingredients_cas 
            ON ingredients(cas);
        """)
        
        # 농도 범위 검색용 복합 인덱스
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_ingredients_conc 
            ON ingredients(conc_unit, conc_basis, conc_min, conc_max);
        """)
        
        # JSONB 동의어 검색용 GIN 인덱스
        cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_ingredients_syn_gin 
            ON ingredients USING GIN (synonym_jsonb jsonb_path_ops);
        """)
    conn.commit()
    print("인덱스 생성 완료")


def init_msds_schema(conn):
    """MSDS 스키마 전체 초기화 (테이블 + 인덱스)"""
    print("=" * 50)
    print("MSDS PostgreSQL 스키마 초기화")
    print("=" * 50)
    
    create_products_table(conn)
    create_ingredients_table(conn)
    create_ingredient_synonyms_table(conn)
    create_indexes(conn)
    
    print("\n모든 테이블 및 인덱스 생성 완료")


def check_tables_exist(conn):
    """생성된 테이블 확인"""
    with conn.cursor() as cursor:
        cursor.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public' 
            AND table_name IN ('products', 'ingredients', 'ingredient_synonyms')
            ORDER BY table_name;
        """)
        tables = cursor.fetchall()
    
    print("\n 생성된 테이블 목록:")
    for table in tables:
        print(f"  - {table[0]}")
    
    return [t[0] for t in tables]

def upsert_product(conn, file_name: str, document_id: str, product_name: str, company_name: str | None = None):
    sql = """
    INSERT INTO products (file_name, document_id, product_name, company_name)
    VALUES (%s, %s, %s, %s)
    ON CONFLICT (file_name, product_name)
    DO UPDATE SET 
        document_id = EXCLUDED.document_id,
        company_name = EXCLUDED.company_name
    RETURNING id;
    """
    with conn.cursor() as cur:
        cur.execute(sql, (file_name, document_id, product_name, company_name))
        row = cur.fetchone()
        if not row:
            raise RuntimeError("RETURNING id 결과가 없습니다")
        pid = row[0]
    conn.commit()
    return pid


def insert_ingredient(conn, product_id: int, ing: dict) -> int:
    """ ingredients에 한 개 성분 삽입 후 id 반환 """
    c = ing.get("concentration") or {}
    syn = ing.get("synonym") or []
    add = ing.get("additional_info") or {}

    sql = """
    INSERT INTO ingredients (
        product_id, name, cas, ec_number,
        conc_raw, conc_value, conc_min, conc_max,
        conc_unit, conc_basis, conc_op_min, conc_op_max,
        synonym_jsonb, additional_info
    ) VALUES (
        %s, %s, %s, %s,
        %s, %s, %s, %s,
        %s, %s, %s, %s,
        %s::jsonb, %s::jsonb
    )
    RETURNING id;
    """
    with conn.cursor() as cur:
        cur.execute(sql, (
            product_id,
            ing.get("name"),
            ing.get("cas"),
            ing.get("ec_number"),
            c.get("raw"),
            c.get("value"),
            c.get("min"),
            c.get("max"),
            c.get("unit"),
            c.get("basis"),
            c.get("op_min"),
            c.get("op_max"),
            json.dumps(syn, ensure_ascii=False),
            json.dumps(add, ensure_ascii=False),
        ))
        ingredient_id = cur.fetchone()[0]
    conn.commit()
    return ingredient_id


def insert_synonyms_rows(conn, ingredient_id: int, synonyms: list[str]):
    """ ingredient_synonyms 테이블에 동의어 다건 삽입 """
    if not synonyms:
        return
    sql = """
    INSERT INTO ingredient_synonyms (ingredient_id, synonym)
    VALUES (%s, %s)
    ON CONFLICT (ingredient_id, synonym) DO NOTHING;
    """
    with conn.cursor() as cur:
        for s in synonyms:
            if s and s.strip():
                cur.execute(sql, (ingredient_id, s.strip()))
    conn.commit()


def save_current_parse_to_postgres(
    conn, 
    md_path: str, 
    section1_result: dict, 
    sec1_text: str, 
    ingredients: list[dict],
    document_id: str = None
) -> int:
    # 1) 제품 upsert
    file_name = Path(md_path).name
    product_name = section1_result.get("product_name")
    company_name = section1_result.get("company_name")
    
    # document_id가 없으면 file_name으로 대체
    if document_id is None:
        document_id = file_name
    
    print(file_name, product_name, company_name, document_id)

    product_id = upsert_product(
        conn,
        file_name=file_name,
        document_id=document_id,  # 추가
        product_name=product_name,
        company_name=company_name
    )

    # 2) 성분 저장
    for it in ingredients:
        ing_id = insert_ingredient(conn, product_id, it)
        insert_synonyms_rows(conn, ing_id, it.get("synonym") or [])

    return product_id




# MSDS Product
## msds_db_regex.py

물질안전보건자료(MSDS/SDS) 텍스트 분석 및 정보 추출을 위한 정규표현식 패턴 모음.

이 모듈은 MSDS/SDS 문서의 다양한 섹션(예: 1장 제품 정보, 2/3장 구성성분)의
헤더를 식별하고, 제품명, 회사 정보, 연락처, 주소 등 특정 정보를 추출하기 위한
정규식들을 정의합니다. 또한, 텍스트 정제에 사용되는 패턴들도 포함합니다.

주요 기능:
- **섹션 헤더 식별**: 한글/영문 SDS의 다양한 표기법을 고려한 섹션 헤더 패턴.
- **정보 라벨 식별**: 제품명, 회사명 등 정보 유형을 나타내는 라벨 패턴.
- **데이터 형식 식별**: CAS 번호, 이메일, URL 등 특정 데이터 형식 패턴.
- **텍스트 정제**: 제어 문자, 불필요한 공백, 페이지 번호 등을 제거하기 위한 패턴.
- **사전 컴파일된 정규식**: 자주 사용되는 패턴들을 `re.compile`로 미리 컴파일하여 성능 최적화.

In [ ]:
# 영문 SDS의 Section 1 헤더(Identification / Product and Company Identification 등)를 다양한 표기 변형으로 매칭하는 패턴 목록.
SEC1_PATTERNS_EN = [
    r"^\s*(?:section\s*)?(?:i|1|01)\s*[:.)\-–—]*\s*(?:identification|product\s+identification|product\s+and\s+company\s+identification)\b",
    r"^\s*0*1\s*[:.)\-–—]*\s*identification\b",
    r"^\s*(?:section\s*1)\s*[:.)\-–—]*\s*identification\b",
]

# 한글 SDS의 1장/섹션 헤더(제품 및 회사에 관한 정보/식별)를 번호·접두 텍스트·문장 구성 변형까지 포함하여 매칭하는 패턴 목록.
SEC1_PATTERNS_KO = [
    r"^\s*(?:##?\s*)?(?:section|섹션)?\s*0*1\s*[:.)\-–—]?\s*(?:화학제품|제품)\s*과?\s*(?:회사|사업장)\s*(?:에\s*관한\s*정보|식별|정보)\s*$",
    r"^\s*(?:##?\s*)?0*1\s*[.)]\s*(?:화학제품|제품)\s*과?\s*(?:회사|사업장)\s*(?:에\s*관한\s*정보|식별|정보)\s*$",
    r"^\s*제?\s*0*1\s*(?:장|부)\s*[:.)\-–—]?\s*(?:화학제품|제품)\s*과?\s*(?:회사|사업장)\s*(?:에\s*관한\s*정보|식별|정보)\s*$",
]

# 최소한 “1:” 같이 숫자+구분자 뒤에 임의 제목이 오는 느슨한 1장 헤더 폴백 매칭용 패턴.
SEC1_PATTERNS_FALLBACK = [
    r"^\s*(?:##?\s*)?0*1\s*[:.)\-–—]\s+.+$",
]

# 한글 SDS의 2장 “유해성 및 위험성”을 엄격한 형태(완전 일치, 문장 끝 고정)로 매칭하는 패턴 목록.
SEC23_PATTERNS_KO_STRICT = [
    r"^\s*(?:##+\s*)?\s*0*2\s*[\.\):\-–—]?\s*유해성\s*(?:및)?\s*위험성\s*$",
    r"^\s*(?:##+\s*)?\s*제?\s*0*2\s*(?:장|부)\s*[\.\):\-–—]?\s*유해성\s*(?:및)?\s*위험성\s*$",
]

# 한글 SDS의 2장 “유해성 및 위험성”을 느슨하게(접미 텍스트 허용) 매칭하는 패턴 목록.
SEC23_PATTERNS_KO_LOOSE = [
    r"^\s*(?:##+\s*)?\s*0*2\s*[\.\):\-–—]?\s*유해성\s*(?:및)?\s*위험성\b",
    r"^\s*(?:##+\s*)?\s*제?\s*0*2\s*(?:장|부)\s*[\.\):\-–—]?\s*유해성\s*(?:및)?\s*위험성\b",
]

# 영문 SDS의 Section 2 Hazard(s) 헤더를 다양한 기입 변형(Section/sect. 02 등)으로 매칭하는 패턴 목록.
SEC23_PATTERNS_EN = [
    r"^\s*(?:##+\s*)?\s*(?:section|sect\.)?\s*0*2\s*[:\.\)\-–—]?\s*(?:hazard|hazards)\b",
]

# 2: 또는 3: 뒤에 임의 제목이 오는 식의 느슨한 2·3장 헤더 폴백 검출용 패턴.
SEC23_PATTERNS_FALLBACK = [
    r"^\s*(?:##+\s*)?\s*[23]\s*[:\.\)\-–—]\s+.+$",
]

# 위의 SEC1 관련 영문/한글/폴백 패턴들을 OR로 합친 컴파일 결과. Section 1 헤더를 한 번에 판별하는 정규식 객체.
SEC1_RE  = re.compile("|".join(SEC1_PATTERNS_EN + SEC1_PATTERNS_KO + SEC1_PATTERNS_FALLBACK), re.I)
# 한글 엄격 패턴 + 영문 패턴을 OR로 합친 컴파일 결과. 2장(유해성) 헤더의 엄격 매칭 용도.
SEC23_RE_STRICT = re.compile("|".join(SEC23_PATTERNS_KO_STRICT + SEC23_PATTERNS_EN), re.I)
# 느슨 한글 + 영문 + 폴백 패턴을 OR로 합친 컴파일 결과. 2/3장 주변 헤더를 느슨하게 탐지.
SEC23_RE_LOOSE  = re.compile("|".join(SEC23_PATTERNS_KO_LOOSE  + SEC23_PATTERNS_EN + SEC23_PATTERNS_FALLBACK), re.I)

# 현재페이지/전체페이지 같은 페이지 네비게이션 라인(예: “2/12”, “3-10”) 제거/무시용 패턴.
PAGE_NAV_PAT      = re.compile(r"^\s*\d+\s*[/\-]\s*\d+\s*$")
# “>>> page 3” 같은 페이지 마커 줄을 걸러내기 위한 패턴.
# PAGE_MARK_PAT     = re.compile(r"^\s*>>>+\s*page\s+\d+\s*$", re.I)
PAGE_MARK_PAT = re.compile(r"^\s*>{2,}\spage[\s_\-]\d+(?:\s*(?:of|/)\s*\d+)?\s*$", re.I)
# 한 줄 전체가 HTML 태그 형태인 경우(예: “<div>”)를 제거하기 위한 패턴.
HTML_TAG_LINE_PAT = re.compile(r"^\s*<[^>]+>\s*$", re.I)

# 제품명/식별자 라벨(영문·한글)의 다양한 표기 후보 리스트. 키-값 추출 시 라벨 인식용.
PRODUCT_LABELS = [
    r"product\s*name", r"product\s*identifier", r"product\s*number", r"product\s*code",
    r"recommended\s*product\s*name", r"trade\s*name",
    r"상품명", r"제품명", r"제품\s*식별자", r"상표명", r"제품\s*명", r"화학제품\s*명", r"물질\s*명",
]

# 회사/제조사/공급자 등 공급 주체 라벨의 다양한 표기 후보 리스트. 연락처/회사 정보 블록 추출 시 라벨 인식용.
COMPANY_LABELS = [
    r"company", r"manufacturer", r"manufactured\s*by", r"supplier", r"distributor", r"importer",
    r"responsible\s*(party|company)", r"producer", r"registrant",
    r"회사명", r"제조사", r"제조업체", r"공급자", r"공급업체", r"수입사", r"수입업체", r"책임회사", r"제조회사", r"판매사", r"판매업체", r"공급처", r"제조원", r"판매원", r"책임판매업자", r"수입원",
]

# product name/제품명 등 제품명 라벨만 단독으로 적힌 안전한 헤더 라인(라벨 단독 라인) 매칭용 정규식.
SAFE_NAME_LABEL = re.compile(
    r"^\s*(product\s*name|product\s*identifier|product\s*number|product\s*code|recommended\s*product\s*name|"
    r"trade\s*name|상품명|제품명|제품\s*식별자|상표명|제품\s*명|화학제품\s*명|물질\s*명)\s*$",
    re.I
)

# company/제조사/공급자… 등 회사 라벨만 단독으로 적힌 안전한 헤더 라인 매칭용 정규식.
SAFE_COMPANY_LABEL = re.compile(
    r"^\s*(company|manufacturer|manufactured\s*by|supplier|distributor|importer|responsible\s*(party|company)|"
    r"producer|registrant|회사명|제조사|제조업체|공급자|공급업체|수입사|수입업체|책임회사|제조회사|판매사|판매업체|공급처|제조원|판매원|책임판매업자|수입원)\s*$",
    re.I
)

# 섹션 타이틀/헤더성 문구 배제용 정규식
HEADER_BAD_FOR_COMPANY = re.compile(
    r'\b(identification|substance|mixture|company/undertaking|details\s+of\s+the\s+supplier|product\s+identifier)\b',
    re.I
)

# 주소 판별에 자주 등장하는 영문/한글 단어 세트(road, st., 대로, 구 등). 주소 탐지 보조용 토큰 목록.
ADDR_WORDS = (
    r"(street|st\.|road|rd\.|avenue|ave\.|blvd\.|drive|dr\.|way|highway|hwy\.|park|suite|ste\.|floor|fl\.|"
    r"building|bldg\.|route|industrial|parkway|pkwy\.|unit|no\.|"
    r"로|길|번길|대로|동|구|군|시|도|읍|면|리|산단|산업단지|단지|지구|번지|호)"
)
# 숫자+지명+주소어 조합, 우편번호/Postal/ZIP/일본식 〒 등 주소 패턴 전반을 포괄적으로 매칭하는 정규식
ADDR_PAT  = re.compile(
    rf"(\d+\s+[A-Za-z가-힣0-9\-]+(?:\s+{ADDR_WORDS})|\b{ADDR_WORDS}\b|\bZIP\b|\bPostal\b|\b〒\b|\b\d{{3}}[-\s]?\d{{3}}\b|\b\d{{5}}(?:-\d{{4}})?\b)",
    re.I
)

# tel/phone/fax/emergency/전화/팩스 등 키워드 또는 국제전화 형식(+국가코드-번호)을 포착하는 전화·팩스 등 연락처 매칭용.
PHONE_PAT = re.compile(r"\b(tel|phone|fax|mobile|emergency|hotline|전화|팩스)\b|\+\d{1,3}[-\s]?\d{1,4}", re.I)

# 연락처 섹션을 가리키는 라벨/키워드 감지(숫자 형식이 없어도 매치).
CONTACT_LABEL_RE = re.compile(
    r'\b(tel|phone|fax|mobile|emergency|hotline|contact|긴급|긴급전화|긴급전화번호|전화|전화번호|연락처)\b',
    re.I
)
# 표준 이메일 주소 형식을 감지하는 정규식.
EMAIL_PAT = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
# http/https/www로 시작하는 URL을 간단 매칭하는 정규식.
WEB_PAT   = re.compile(r"(https?://|www\.)\S+", re.I)

# 저작권/배포 제한/면책 고지 등 문서 하단 저작권 관련 문구를 식별해 본문 추출에서 배제하기 위한 키워드 패턴.
COPYRIGHT_STOPWORDS = re.compile(
    r"(저작권|판권|배포|복사|다운로드|허용|동의|재판매|재산상|이득|규제|면제|예외|"
    r"copyright|all\s+rights\s+reserved|distribution|redistribution|reproduce|copy|download|permitted|consent|resale|profit|disclaimer|exception|exempt|regulation)",
    re.I
)

# SDS/MSDS 문서 제목 라인(“SAFETY DATA SHEET”, “물질안전보건자료” 등)을 감지해 메타 타이틀을 본문 처리에서 분리하기 위한 패턴.
DOC_TITLE_STOP = re.compile(
    r"(제품안전취급서|물질안전보건자료서?|MSDS|SDS|MATERIAL\s+SAFETY\s+DATA\s+SHEET|SAFETY\s+DATA\s+SHEET)",
    re.I
)

# 영문 법인 접미사(Inc., Ltd., GmbH, LLC 등)나 한글 법인 형태(주식회사, (주) 등)를 포함한 회사명 후보를 폭넓게 매칭하는 정규식.
COMPANY_CAND_PAT = re.compile(
    r"((?:[A-Z][A-Za-z&.,\-\s]{1,60}\b(?:Co\.?,?\s*Ltd\.?|Company|Corporation|Corp\.?|Inc\.?|LLC|GmbH|"
    r"S\.?A\.?|S\.?p\.?A\.?|Ltd\.?|LP|LLP|BV|NV|KK))|"
    r"(?:주식회사|유한회사|유한책임회사|합자회사|합명회사|\(주\))\s*[A-Za-z가-힣0-9&.\-\s]{1,40}|"
    r"(?:[A-Za-z가-힣0-9&.\-\s]{1,40}\s*(?:주식회사|유한회사|유한책임회사|\(주\)))|"
    r"(?:3M|Shell|Praxair)\s*(?:Company|Corp\.?|Inc\.?|Korea\s*Co\.?,?\s*Ltd\.?)?)",
    re.I
)

# “CAS No.”, “UN No.”, “EU No.”, “KE No.” 같은 코드 라벨 키워드 감지용 정규식.
CODE_LABEL_RX = re.compile(r"\b(CAS|UN|EU|KE)\s*No\.?\b", re.I)
# 실제 코드 값 포맷(CAS 00000-00-0, KE-xxxxxx, 단순 숫자 코드 등)을 검증/추출하기 위한 값 패턴 정규식.
CODE_VALUE_RX = re.compile(r"^(?:KE-\d+|\d{2,7}-\d{2}-\d|\d{3,5}|\d{3}-\d{4})$", re.I)


# 제어문자 및 특수 공백 제거를 위한 패턴
# CONTROL_WS = re.compile(r"[\u0000-\u0009\u000B-\u000C\u000E-\u001F\u007F\u00A0\u2000-\u200B\u2028\u2029\uFEFF]")
CONTROL_WS = re.compile(r"[\u0000-\u0009\u000B-\u000C\u000E-\u001F\u007F\u00A0\u00AD\u2000-\u200B\u2028\u2029\u2060\uFEFF]")
# 점(·•‧∙●∘・｡。．)과 유사한 문자를 모두 '.'으로 통일하기 위한 매핑 테이블
DOT_LIKE = dict.fromkeys(map(ord, "·•‧∙●∘・｡。．"), ord('.'))

# 전각/호환 문자를 ASCII로 통일하기 위한 매핑
FULLWIDTH = {
    ord("（"): ord("("), ord("）"): ord(")"),
    ord("："): ord(":"),
    ord("．"): ord("."),
    ord("－"): ord("-"), ord("—"): ord("-"), ord("–"): ord("-"),
}

# 헤더/식별 배제
HEADER_EXCLUDE_RE = re.compile(r'\b(section|identifier|identification|product\s+identifier)\b', re.I)

# 1.x 서브섹션 감지 하위 절 중 "1.x"로 시작하는 섹션 번호만 감지.
SUBSEC_1X_RE = re.compile(r'^\s*(?:#{1,6}\s*)?1\.\d+\b', re.I)

# Markdown 헤더 라인
MD_HEADER_RE = re.compile(r'^\s*#{1,6}\s+')

# Section 1 내에서 종료 힌트 라벨
SEC1_END_HINT_RE = re.compile(r'\b(emergency\s+telephone|manufacturer|supplier)\b', re.I)



#######Ingredients 단독######


# 구분자
# 헤더 번호 뒤에 따라오는 대표 구분자 집합 (: . ) - – —)
DASH_CLASS = r"[:\.\)\-\u2013\u2014]"
# 헤더 내부/사이 구분자(공백 포함) 확장 집합
SEP = r"[:\s\.\)\-\u2013\u2014]"

# 루트 헤더
# "section N" (마크다운 해시 허용), "N{구분자}", "로마숫자{구분자}" 패턴
# N은 1~16, 로마숫자는 i~xvi까지 지원
ROOT_ANY_RX = re.compile(
    rf"^\s*(?:(?:##+\s*)?section\s*(?:[1-9]|1[0-6]|i|ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii|xiii|xiv|xv|xvi)\s*{DASH_CLASS}?|"
       r"(?:##+\s*)?(?:[1-9]|1[0-6])\s*{DASH_CLASS}|"
       r"(?:##+\s*)?(?:i|ii|iii|iv|v|vi|vii|viii|ix|x|xi|xii|xiii|xiv|xv|xvi)\s*{DASH_CLASS})",
    re.I
)

# 서브섹션 탐지
# "3.2", "3.2.", "3 . 2", "3.2.1" 등 변형 포함
SUBSEC_RX = re.compile(r"^\s*(?:##+\s*)?\d+\s*\.\s*\d+(?:\s*\.\s*\d+)*\s*[.)]?\b", re.I)

# 헤더 힌트(EN): 성분/조성 영역을 지칭하는 영문 표현
HEAD_HINTS_EN = [
    r"composition\s*/\s*information\s*on\s*ingredients",
    r"composition\s*and\s*information\s*on\s*ingredients",
    r"composition\s*,\s*information\s*on\s*ingredients",
    r"composition\s*information\s*on\s*ingredients",
    r"information\s*on\s*ingredients",
    r"ingredients",
    r"data\s*on\s*components",
    r"components",
]
# 헤더 힌트(KO): 성분/조성 관련 한글 표현
HEAD_HINTS_KO = [
    r"구성성분의\s*명칭\s*및\s*함유량",
    r"구성성분의\s*명칭\s*및\s*조성",
    r"구성\s*성분",
    r"성분",
    r"조성",
]
# 성분/조성 헤더 힌트 통합 정규식
HEAD_HINTS_RE = re.compile("|".join([rf"\b{h}\b" for h in HEAD_HINTS_EN] + HEAD_HINTS_KO), re.I)

# 루트 폴백 힌트: 섹션 분류 키워드(EN/KO)
FALLBACK_ROOT_HINTS = re.compile(
    r"(first[-\s]?aid|응급조치|응급조치요령|handling|storage|취급|저장|"
    r"physical|물리화학|stability|안정성|reactivity|반응성|toxicological|독성|ecological|환경|"
    r"disposal|폐기|transport|운송|regulatory|법적|other|기타)", re.I)


# 정규화

# 허용 단위 화이트리스트(정규화 이후 기준)
UNIT_WHITELIST = {"%", "ppm", "ppb", "mg/L", "g/L", "mg/kg", "g/kg"}

# basis(혼합 기준) 추론을 위한 표현 패턴 목록과 정규화 결과 매핑
BASIS_PATTERNS = [
    (re.compile(r"\b(w/?w)\b", re.I), "w/w"),
    (re.compile(r"\b(v/?v)\b", re.I), "v/v"),
    (re.compile(r"\bwt\s*%\b", re.I), "w/w"),
    (re.compile(r"(중량\s*%|w/w\s*%)", re.I), "w/w"),
    (re.compile(r"(부피\s*%|v/v\s*%)", re.I), "v/v"),
    (re.compile(r"%\s*\(\s*w\s*/\s*w\s*\)", re.I), "w/w"),
    (re.compile(r"%\s*\(\s*v\s*/\s*v\s*\)", re.I), "v/v"),
]

# 동의어 토큰 분할용 구분자(쉼표/세미콜론/슬래시/수직바/개행)
DELIMS_RX = re.compile(r"[;,/|\n]+")

# 혼합물 키워드(한/영) 감지
KO_MIX_RX = re.compile(r"^\s*혼합물\s*$", re.I)
EN_MIX_RX = re.compile(r"^\s*mixture\s*$", re.I)

## msds_db_text_norm.py

[products table 관련]
MSDS 텍스트 분석을 위한 텍스트 정규화 및 정제 유틸리티 모듈.

이 모듈은 MSDS(물질안전보건자료) 문서의 텍스트를 분석하기 전에,
다양한 형태의 '노이즈'를 제거하고 문자열을 일관된 표준 형식으로
변환하는 헬퍼 함수들을 제공합니다.

주요 기능:
- `norm`: 제어 문자, 특수 공백, 서식 문자 등을 제거하고 소문자로 변환하는
  범용 정규화 함수입니다.
- `norm_company`: `norm`에 더해, 회사명 추출에 특화된 정제 로직(괄호 제거,
  연락처 정보 분리 등)을 수행합니다.
- `is_noise_line`: 특정 라인이 페이지 번호나 HTML 태그와 같은 무시해도
  되는 노이즈인지를 판별합니다.
- `safe_value_after_label`: "라벨: 값" 형식의 라인에서 안전하게 값을
  추출합니다.
- `clean_section1_lines`: 섹션 1 텍스트 블록에서 저작권 문구나 문서 제목 등
  불필요한 라인들을 제거합니다.

In [ ]:
# 문자열을 표준 형태로 정규화 함수
def norm(s: str) -> str:
    t = CONTROL_WS.sub(" ", s)
    t = t.translate(DOT_LIKE)
    t = re.sub(r"[*_`]+", "", t)
    t = t.replace("\t", " ")
    t = t.replace("：", ":").replace("－", "-").replace("—", "-").replace("–", "-")
    t = t.strip(" \t:;,-|")
    t = re.sub(r"\s+", " ", t)
    return t

# 회사명 문자열 정규화 전용 함수
def norm_company(s: str) -> str:
    t = norm(s)
    t = re.sub(r"\([^)]*\)", "", t).strip()
    t = re.split(r"(?:Tel\.?|Phone|Fax|E-?mail|Email|www\.|https?://|주소|전화|팩스)\b", t, maxsplit=1)[0].strip()
    t = re.sub(r"[|•]+", " ", t)
    t = re.sub(r"\s{2,}", " ", t).strip()
    return t

# 입력된 한 줄이 '잡음라인'인지 판단하는 함수
def is_noise_line(s: str) -> bool:
    t = s.strip()
    if not t: return True
    if PAGE_NAV_PAT.match(t) or PAGE_MARK_PAT.match(t): return True
    if HTML_TAG_LINE_PAT.match(t): return True
    return False

# 라벨 뒤의 값을 안전하게 추출하기 위한 헬퍼 함수
def safe_value_after_label(line: str, safe_label_re: re.Pattern) -> Optional[str]:
    m = re.split(r"[:\-–]\s*", line, maxsplit=1)
    if len(m) == 2:
        left, right = norm(m[0]), norm(m[1])
        if safe_label_re.match(left) and right:
            return right
    return None

# Section 1 (제품 및 회사에 관한 정보) 블록에서 불필요한 라인을 걸러내는 후처리 함수
def clean_section1_lines(lines: List[str]) -> List[str]:
    out = []
    for ln in lines:
        t = ln.strip()
        if not t: continue
        if COPYRIGHT_STOPWORDS.search(t): continue
        if DOC_TITLE_STOP.search(t): continue
        if t.startswith("| ---"): continue
        if re.match(r"^\|\s*[^|]+\s*\|$", t): continue
        if CODE_LABEL_RX.search(t): continue
        out.append(ln)
    return out

## msds_db_section1_slicer.py

[products table 관련]
MSDS 텍스트에서 섹션 1(제품 및 회사 정보)을 슬라이싱하는 모듈.

이 모듈은 MSDS(물질안전보건자료)의 전체 텍스트로부터 섹션 1에 해당하는
부분을 정확히 잘라내는 함수를 제공합니다.

주요 기능:
- `slice_section1`: 'Section 1' 헤더를 탐지하여 시작점을 찾고,
  'Section 2' 또는 'Section 3' 헤더가 나타나는 지점을 종료 지점으로 삼아
  텍스트를 슬라이싱합니다. 또한, 추출된 텍스트에서 불필요한 노이즈를
  제거하는 정제 작업도 수행합니다.
- `slice_section1_debug`: `slice_section1`과 동일한 작업을 수행하면서,
  슬라이싱 과정에서 사용된 인덱스 등 디버깅 정보를 함께 반환합니다.

In [ ]:
# 다음 주요 섹션의 시작 라인을 탐색하기 위한 함수
def find_next_section2_or_3(lines: List[str], start_idx: int) -> Optional[int]:
    for k in range(start_idx + 1, min(len(lines), start_idx + 4000)):
        row = norm(lines[k])
        if SEC23_RE_STRICT.search(row):
            return k
    for k in range(start_idx + 1, min(len(lines), start_idx + 4000)):
        row = norm(lines[k])
        if SEC23_RE_LOOSE.search(row):
            return k
    return None


# Section 1 본문만 추출하는 메인 함수.
def slice_section1(text: str, max_lookahead_lines: int = 1200, body_limit: int = 1200) -> str:
    lines = text.splitlines()
    # 1. Section 1 헤더 탐색
    start_idx = None
    for i, ln in enumerate(lines[:max_lookahead_lines]):
        if SEC1_RE.search(norm(ln)):
            start_idx = i
            break
    if start_idx is None:
        start_idx = 0

    # 2. 실질적인 본문 시작라인 찾기 (공백/잡음 라인 통과)
    j = start_idx + 1
    while j < len(lines):
        if norm(lines[j]) and not is_noise_line(lines[j]):
            break
        j += 1
    start_idx = min(j, len(lines)-1)

    # 3. 다음 섹션 2 또는 3의 시작 인덱스 탐색
    next_idx = find_next_section2_or_3(lines, start_idx)
    end_idx = next_idx if next_idx is not None else min(len(lines), start_idx + body_limit)

    # 4. 라인 슬라이스 및 정제
    raw = lines[start_idx:end_idx]
    body = [ln for ln in raw if not is_noise_line(ln)]
    body = clean_section1_lines(body)
    return "\n".join(body)


# slice_section1()의 디버그 버전.
def slice_section1_debug(text: str, max_lookahead_lines: int = 1200, body_limit: int = 1200):
    lines = text.splitlines()

    # Section 1 헤더 탐색
    start_idx = None
    for i, ln in enumerate(lines[:max_lookahead_lines]):
        if SEC1_RE.search(norm(ln)):
            start_idx = i
            break
    if start_idx is None:
        start_idx = 0

    # 실질적인 본문 시작라인 보정
    j = start_idx + 1
    while j < len(lines):
        if norm(lines[j]) and not is_noise_line(lines[j]):
            break
        j += 1
    start_idx = min(j, len(lines)-1)

    # 다음 섹션 탐색
    next_idx = find_next_section2_or_3(lines, start_idx)
    end_idx = next_idx if next_idx is not None else min(len(lines), start_idx + body_limit)

    # Section 1 본문 추출 및 정제
    raw = lines[start_idx:end_idx]
    body = [ln for ln in raw if not is_noise_line(ln)]
    body = clean_section1_lines(body)
    section_text = "\n".join(body)

    # 디버그 정보 구성
    debug = {"start_idx": start_idx, "end_idx": end_idx, "found_next": next_idx}
    return section_text, debug

## msds_db_section1_extractors.py

[products table 관련]
MSDS 문서의 섹션 1에서 제품명 및 회사명 정보를 추출하는 모듈.

이 모듈은 MSDS(물질안전보건자료) 문서의 텍스트로부터 제품명과 회사명을
찾아내는 함수들을 제공합니다. 다양한 형식의 문서에 대응하기 위해
라벨 기반 탐색, 패턴 매칭, 근접도 점수화 등 여러 휴리스틱 기법을 사용합니다.

주요 기능:
- `collect_product_candidates`: 'Product Name', '제품명' 등의 라벨을
  단서로 문서에서 제품명 후보들을 수집합니다.
- `collect_company_candidates`: 'Company', '제조사' 등의 라벨과
  '(주)', 'Ltd.' 같은 법인 형태 패턴을 이용해 회사명 후보들을 수집합니다.
- `pick_company_weighted_with_label_proximity`: 수집된 회사명 후보들 중,
  라벨과의 근접도 등 여러 요소를 점수화하여 가장 가능성 높은 회사명을 선택합니다.
- `_contains_only_contact_info`: 특정 라인이 주소나 전화번호 같은 연락처
  정보인지를 판별하여 제품명/회사명 후보에서 제외하는 헬퍼 함수입니다.


In [ ]:
def _contains_only_contact_info(text: str) -> bool:
    # 0) 라벨 키워드만으로도 즉시 연락처로 판정
    if CONTACT_LABEL_RE.search(text):
        return True

    has_contact = any(p.search(text) for p in (ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT))
    if not has_contact:
        return False

    # 의미 있는 단어 수가 매우 적으면 연락처/주소 전용으로 판단
    meaningful_words = re.findall(r'[A-Za-z가-힣]{2,}', text)
    if len(meaningful_words) <= 2:
        return True

    # 숫자 비중이 과한 경우도 연락처로 판단
    digits = sum(ch.isdigit() for ch in text)
    if digits >= max(6, len(text) // 3):
        return True

    # 주소가 길게 포함된 라인
    if len(text) > 40 and ADDR_PAT.search(text):
        return True

    return False


def collect_product_candidates(lines: List[str]) -> List[str]:
    """제품명 후보 수집"""
    out: List[str] = []

    for i, ln in enumerate(lines[:300]):
        if not any(re.search(lbl, ln, flags=re.I) for lbl in PRODUCT_LABELS):
            continue

        # 1) 라벨+값 한 줄 처리
        v = safe_value_after_label(ln, SAFE_NAME_LABEL)
        if v:
            v2 = norm(v)
            if v2 and not any(p.search(v2) for p in [ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT]):
                if not _contains_only_contact_info(v2):
                    out.append(v2)
                    continue

        # 2) 같은 줄 콜론/탭/공백 분리
        same = (re.search(r"[:\-–]\s*(.+)$", ln) or
                re.search(r"\t+(.+)$", ln) or
                re.search(r"\s{3,}(.+)$", ln))
        if same:
            v2 = norm(same.group(1))
            if v2 and not any(p.search(v2) for p in [ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT]):
                if not _contains_only_contact_info(v2):
                    out.append(v2)

        # 3) 다음 줄 hop 탐색
        for hop in range(1, 8):
            if i + hop >= len(lines):
                break
            raw = lines[i + hop]

            if MD_HEADER_RE.match(raw):
                continue

            cand = norm(raw)
            if not cand or is_noise_line(raw):
                continue
            if any(re.search(lbl, cand, flags=re.I) for lbl in PRODUCT_LABELS):
                continue
            if any(p.search(cand) for p in [ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT]):
                continue
            if _contains_only_contact_info(cand):
                continue
            out.append(cand)
            break

    # 중복 제거 및 상위 5개
    uniq, seen = [], set()
    for c in out:
        cc = c.strip()
        if cc and cc.lower() not in seen:
            seen.add(cc.lower())
            uniq.append(cc)
    return uniq[:5]


def collect_company_candidates(lines: List[str]) -> List[str]:
    """회사명 후보 수집"""
    out: List[str] = []

    # 1단계: 라벨 기반 수집
    for i, ln in enumerate(lines[:300]):
        if not any(re.search(lbl, ln, flags=re.I) for lbl in COMPANY_LABELS):
            continue

        # 1) 라벨+값 한 줄 처리 (라벨 접두 제거)
        v = safe_value_after_label(ln, SAFE_COMPANY_LABEL)
        if v:
            vv = norm_company(v)
            vv = re.sub(r'^\s*(?:' + '|'.join(COMPANY_LABELS) + r')\s*[:\-–]?\s*', '', vv, flags=re.I)
            if vv and not any(p.search(vv) for p in [ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT]):
                if not _contains_only_contact_info(vv):
                    out.append(vv)
                    continue

        # 2) 같은 줄 콜론/탭/공백 분리
        same = (re.search(r"[:\-–]\s*(.+)$", ln) or
                re.search(r"\t+(.+)$", ln) or
                re.search(r"\s{3,}(.+)$", ln))
        if same:
            vv = norm_company(same.group(1))
            if vv and not any(p.search(vv) for p in [ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT]):
                if not _contains_only_contact_info(vv):
                    out.append(vv)

        # 3) 다음 줄 hop 탐색
        for hop in range(1, 8):
            if i + hop >= len(lines):
                break
            raw = lines[i + hop]

            if MD_HEADER_RE.match(raw):
                continue

            cand = norm_company(raw)
            if not cand or is_noise_line(raw):
                continue
            if any(re.search(lbl, cand, flags=re.I) for lbl in COMPANY_LABELS):
                continue
            if any(p.search(cand) for p in [ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT]):
                continue
            if _contains_only_contact_info(cand):
                continue
            out.append(cand)
            break

    # 2단계: 회사명 패턴(법인 접미사) 기반 추가 수집
    for i, ln in enumerate(lines[:120]):
        if MD_HEADER_RE.match(ln):
            continue
        t = norm_company(ln)
        if not t or is_noise_line(ln):
            continue
        if _contains_only_contact_info(t):
            continue
        m = COMPANY_CAND_PAT.search(t)
        if m:
            out.append(m.group(0).strip())

    # 3단계: 헤더성 문구 + 연락처 라인 제거
    out = [c for c in out if not HEADER_BAD_FOR_COMPANY.search(c)]
    out = [c for c in out if not any(p.search(c) for p in (ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT))]
    out = [c for c in out if not _contains_only_contact_info(c)]

    # 중복 제거 및 상위 5개
    uniq, seen = [], set()
    for c in out:
        cc = c.strip()
        if cc and cc.lower() not in seen:
            seen.add(cc.lower())
            uniq.append(cc)
    return uniq[:5]


def pick_company_weighted_with_label_proximity(lines: List[str], candidates: List[str]) -> Optional[str]:
    """회사명 후보 중 최종 선택 (라벨 근접도 가중)"""
    if not candidates:
        return None

    nlines = [norm_company(x) for x in lines]

    # 라벨 위치 인덱스 수집
    label_idx = [
        i for i, ln in enumerate(nlines)
        if SAFE_COMPANY_LABEL.match(ln) or any(re.search(lbl, ln, re.I) for lbl in COMPANY_LABELS)
    ]

    def score_of(cand: str) -> float:
        s = 0.0

        # 1) 라벨 바로 아래 줄 강가중
        for li in label_idx:
            for off, w in [(1, 2.8), (2, 1.4), (3, 0.7)]:
                idx = li + off
                if 0 <= idx < len(nlines) and cand == nlines[idx]:
                    s += w

        # 2) 헤더성 문구 감점
        if HEADER_BAD_FOR_COMPANY.search(cand):
            s -= 1.5

        # 3) 주소/연락처 강한 감점
        if any(p.search(cand) for p in (ADDR_PAT, PHONE_PAT, EMAIL_PAT, WEB_PAT)):
            s -= 1.5

        # 4) 연락처만 있는 줄 강한 감점 (추가 안전장치)
        if _contains_only_contact_info(cand):
            s -= 2.0

        # 5) 회사명 형태 소폭 가점
        if COMPANY_CAND_PAT.search(cand):
            s += 0.8

        return s

    return max(candidates, key=score_of) or None


## msds_db_section1_pipeline.py

[products table 관련]
MSDS 섹션 1 분석 및 정보 추출을 위한 메인 파이프라인 모듈.

이 모듈은 MSDS(물질안전보건자료)의 전체 텍스트로부터 섹션 1에 해당하는
부분을 잘라내고, 그 안에서 제품명과 회사명을 추출하는 함수를 제공합니다.

주요 기능:
- `extract_section1_and_fields_from_text`: 전체 텍스트를 입력받아
  내부적으로 슬라이싱, 후보 수집, 최종 선택 단계를 모두 수행하여
  최종 제품명과 회사명, 그리고 분석 과정에서 수집된 후보 목록을 반환합니다.

의존성:
- `msds_db_section1_slicer`: 텍스트에서 섹션 1 부분을 잘라내는 역할.
- `msds_db_section1_extractors`: 잘라낸 텍스트에서 제품명 및 회사명
  후보를 수집하고, 최종 값을 선택하는 역할.

In [ ]:
def extract_section1_and_fields_from_text(text: str):
    section1 = slice_section1(text)
    lines = section1.splitlines()

    prodcands = collect_product_candidates(lines)
    compcands = collect_company_candidates(lines)

    product_name = prodcands[0] if prodcands else 'UNKNOWN'
    company_name = pick_company_weighted_with_label_proximity(lines, compcands) or 'UNKNOWN'

    return {
        'product_name': product_name,
        'company_name': company_name,
        'section1_excerpt': section1[:1500],
        'product_candidates': prodcands,
        'company_candidates': compcands,
    }


# MSDS Ingredients
## msds_db_ingredients_norm.py

[ingredients table 관련]
텍스트 분석을 위한 문자열 정규화(Normalization) 모듈.

이 모듈은 텍스트 비교 및 검색의 정확도를 높이기 위해 문자열을 표준 형식으로
변환하는 함수들을 제공합니다. 또한, 텍스트의 구조적 특징(예: 마크다운 테이블)을
판별하는 헬퍼 함수도 포함합니다.

주요 기능:
- `norm_lower`: 제어 문자, 특수 공백, 마크다운 서식 등을 제거하고
  모든 문자를 소문자로 변환하여 '검색 친화적인' 표준 형식으로 만듭니다.
- `is_table_line_raw`: 주어진 라인이 마크다운 테이블의 행 형식인지 판별합니다.

In [ ]:
# 문자열을 '검색/비교 친화적 소문자 표준형'으로 정규화
def norm_lower(s: str) -> str:
    """
    문자열을 '검색 및 비교 친화적인' 소문자 표준 형식으로 정규화합니다.

    이 함수는 다음과 같은 정규화 단계를 순차적으로 수행합니다:
    1. 제어 문자(`CONTROL_WS`)를 공백으로 변환합니다.
    2. 점과 유사한 유니코드 문자(`DOT_LIKE`)를 표준 온점('.')으로 통일합니다.
    3. 전각 문자(`FULLWIDTH`)를 해당하는 반각 문자로 변환합니다.
    4. 다양한 유니코드 공백 문자를 표준 공백(' ')으로 바꿉니다.
    5. 마크다운 서식 문자('*', '_', '`')를 제거합니다.
    6. 탭 문자를 공백으로 변환합니다.
    7. 문자열 양 끝의 공백, 탭, 및 특정 구분자들을 제거합니다.
    8. 연속된 여러 공백을 단일 공백으로 축약합니다.
    9. 최종적으로 모든 문자를 소문자로 변환합니다.

    Args:
        s (str): 정규화할 원본 문자열.

    Returns:
        str: 정규화된 소문자 문자열.
    
    Example:
        >>> norm_lower("  *Hello*　World！  ")
        'hello world!'
    """
    t = CONTROL_WS.sub(" ", s)
    t = s.translate(DOT_LIKE).translate(FULLWIDTH)
    t = re.sub(r"[\u2000-\u200B\u2060\u00A0\u00AD]", " ", t)
    t = re.sub(r"[*_`]+", "", t)
    t = t.replace("\t", " ")
    t = t.strip(" \t:;,-|")
    t = re.sub(r"\s+", " ", t)
    return t.lower()

# Markdown 표 형태의 라인 여부를 판별
def is_table_line_raw(ln: str) -> bool:
    """
    주어진 라인이 마크다운(Markdown) 테이블의 행 형식인지 판별합니다.

    판별 기준은 다음과 같습니다:
    - 라인의 양 끝 공백을 제거했을 때, '|'로 시작하고 '|'로 끝나야 합니다.
    - 라인에 포함된 '|' 문자가 2개 이상이어야 합니다 (최소 1개 열).

    Args:
        ln (str): 검사할 한 줄의 문자열.

    Returns:
        bool: 마크다운 테이블 행 형식이면 True, 아니면 False.

    Example:
        >>> is_table_line_raw("| Column 1 | Column 2 |")
        True
        >>> is_table_line_raw("  | A | B |  ")
        True
        >>> is_table_line_raw("This is not a table line.")
        False
        >>> is_table_line_raw("| Incomplete ")
        False
    """
    s = ln.strip()
    return s.startswith("|") and s.endswith("|") and s.count("|") >= 2

## msds_db_ingredients_locator.py

[ingredients table 관련]
MSDS 텍스트에서 구성성분 섹션의 경계를 탐지하는 로케이터 모듈.

이 모듈은 MSDS(물질안전보건자료) 문서의 전체 텍스트(줄 단위 리스트)를
입력받아, 구성성분 정보가 포함된 섹션(주로 2장 또는 3장)의 시작과 끝
인덱스를 찾는 함수들을 제공합니다.

다양한 형식의 문서를 처리하기 위해 여러 단계의 탐지 전략(엄격, 완화, 폴백)과
휴리스틱을 사용합니다.

주요 기능:
- `collect_root_index_*`: 'Section 1', '2.' 등 주요 섹션 헤더의 위치를 탐지.
- `select_start_for_composition`: 여러 단서를 조합하여 구성성분 섹션의
  시작점을 결정.
- `find_next_root_after`: 현재 섹션 번호보다 큰 번호의 섹션이 나타나는
  지점을 찾아 섹션의 끝으로 간주.
- `fallback_find_next_by_title`: 숫자 번호가 없는 경우, 제목 키워드를
  기반으로 다음 섹션의 시작점을 추정.

In [ ]:
# 가까운 인덱스 묶음을 간소화하여 대표 인덱스만 유지
def _dedup_close(idxs: List[int], gap: int = 3) -> List[int]:
    out=[]
    for x in sorted(idxs):
        if not out or x - out[-1] > gap:
            out.append(x)
    return out

# 루트 헤더(숫자/로마/Section) 엄격 탐지
def collect_root_index_strict(lines: List[str]) -> List[int]:
    idxs=[]
    for i, ln in enumerate(lines):
        if is_table_line_raw(ln): continue
        t = norm_lower(ln)
        if SUBSEC_RX.search(t): continue  # 선제 배제
        if ROOT_ANY_RX.search(t):
            idxs.append(i)
    return _dedup_close(idxs)

# 숫자 기반 완화 탐지(섹션 키워드 optional)
def collect_root_index_relaxed(lines: List[str]) -> List[int]:
    idxs=[]
    for i, ln in enumerate(lines):
        t = norm_lower(ln)
        if SUBSEC_RX.search(t): continue
        if re.search(r"(?:^|[\s])(section\s*)?(?:[1-9]|1[0-6])\s*[:\.\)\-\u2013\u2014]", t, re.I):
            idxs.append(i)
    return _dedup_close(idxs)

# 성분/조성 헤더 힌트(EN/KO) 탐지
def find_composition_headers(lines: List[str]) -> List[int]:
    idxs=[]
    for i, ln in enumerate(lines):
        if HEAD_HINTS_RE.search(norm_lower(ln)):
            idxs.append(i)
    return _dedup_close(idxs, gap=2)

# 조합 규칙으로 ‘성분/조성’ 시작 인덱스 선택
def select_start_for_composition(lines: List[str], roots: List[int], prefer_numbers=(3,2)) -> Optional[int]:
    comp_idxs = find_composition_headers(lines)
    # 1) '3' + 힌트 최우선
    for i in comp_idxs:
        if re.match(r"^\s*(?:##+\s*)?(?:section\s*)?3\s*[:\.\)\-\u2013\u2014]\b", norm_lower(lines[i])):
            return i
    # 2) 힌트만 만족
    if comp_idxs:
        return comp_idxs[0]
    # 3) roots와 힌트가 동시에 일치
    for pn in prefer_numbers:
        num_pat = re.compile(rf"^\s*(?:##+\s*)?(?:section\s*)?{pn}\s*{DASH_CLASS}\b", re.I)
        for i in roots:
            hdr = norm_lower(lines[i])
            if num_pat.search(hdr) and HEAD_HINTS_RE.search(hdr):
                return i
    # 4) roots 중 힌트만 일치
    for i in roots:
        if HEAD_HINTS_RE.search(norm_lower(lines[i])):
            return i

    # 5) roots 첫 번째 또는 None 최종 폴백
    return roots[0] if roots else None


# 루트 헤더 라인에서 '장 번호(1~2자리 정수)'를 추출하기 위한 보조 파서.
def parse_root_number(t: str) -> Optional[int]:
    # 1) Markdown/리스트 프리픽스 제거
    t2 = re.sub(r"^[#>*\s]+", "", t)
    # 2) "Section <num> <sep>" 또는 "<num> <sep>" 매칭
    m = re.search(rf"^\s*(?:section\s*)?(\d{{1,2}})\s*{SEP}\b", t2, re.I) or \
        re.search(rf"(?:^|[\s])(\d{{1,2}})\s*{SEP}\b", t2)
    # 3) 성공 시 정수 변환
    return int(m.group(1)) if m else None

# 현재 장 번호(n_curr)의 하위 소절인지 여부를 판정
def is_subsection_of(n_curr: int, line_norm: str) -> bool:
    # 같은 섹션의 소절(예: "3.2", "3 . 2", "3.2.1")
    # line_norm 사전 정규화(norm_lower)된 문자열을 가정.
    return bool(re.match(rf"^\s*(?:##+\s*)?{n_curr}\s*\.\s*\d", line_norm))

# start_i 이후의 루트 후보 인덱스 중 '상위 번호 증가'가 발생하는 첫 지점을 반환
def find_next_root_after(lines: List[str], roots: List[int], start_i: int, min_span_lines: int = 8) -> Optional[int]:
    # 시작 인덱스 라인의 정규화 텍스트에서 현재 루트 장 번호를 파싱
    curr_n = parse_root_number(norm_lower(lines[start_i]))
    # 미리 수집된 루트 후보 인덱스들을 순회
    for i in roots:
        # 시작 인덱스 이전 또는 같은 위치는 건너뜀
        if i <= start_i: 
            continue
        # 시작점으로부터 너무 가까운 후보는 노이즈 가능성이 있어 건너뜀
        if i - start_i < min_span_lines:
            continue
        # 후보 라인을 정규화하여 다음 루트 번호 파싱 준비
        tline = norm_lower(lines[i])
        # 후보 라인에서 다음 루트 장 번호를 파싱
        nxt_n = parse_root_number(tline)
        if curr_n is not None and nxt_n is not None:
            # 같은 장의 소절은 내부 포함
            if nxt_n == curr_n and (SUBSEC_RX.search(tline) or is_subsection_of(curr_n, tline)):
                continue
            # 번호 감소/동일 루트는 컷 아님
            if nxt_n <= curr_n:
                continue
            # 번호 증가 → 다음 루트 경계
            return i
        # 숫자 파싱 실패 케이스는 폴백 로직에서 처리
    return None

# 폴백: 제목 힌트를 활용해 다음 루트 경계를 추정.
def fallback_find_next_by_title(lines: List[str], start_i: int, scan_ahead: int = 4000) -> Optional[int]:
    best=None
    for k in range(start_i + 8, min(len(lines), start_i + scan_ahead)):
        raw = lines[k]; t = norm_lower(raw)
        # 서브섹션(3.2, 3.2.1 등)으로 보이는 라인은 건너뜀
        if SUBSEC_RX.search(t): 
            continue
        
        if FALLBACK_ROOT_HINTS.search(t):
            score = 0
            # +4: 마크다운 헤딩(##) 외양
            if re.match(r"^\s*##+\s*\S", raw): score += 4
            # +1: 짧은 제목 길이(<80자)
            if len(raw) < 80: score += 1
            # -3: 문장부호(마침표/느낌표/물음표)로 끝남 → 문장일 가능성
            if re.search(r"[.!?]\s*$", raw): score -= 3
            # -1: 콤마 2개 초과 → 열거형 문장일 가능성
            if raw.count(",") > 2: score -= 1
            # -1: 너무 긴 라인(>140자) → 본문일 가능성
            if len(raw) > 140: score -= 1

            if best is None or score > best[0]:
                best = (score, k)
    return best[1] if best else None



## msds_db_ingredients_slicer.py

[ingredients table 관련]
MSDS 텍스트에서 구성성분 섹션을 슬라이싱(Slicing)하는 모듈.

이 모듈은 MSDS(물질안전보건자료)의 전체 텍스트로부터 구성성분 정보가
포함된 섹션(주로 2장 또는 3장)을 정확히 잘라내는 기능을 제공합니다.

주요 기능:
- `slice_section_2_or_3`: `msds_db_ingredients_locator` 모듈의 여러
  탐지 함수들을 조합하여, 구성성분 섹션의 시작과 끝을 결정하고 해당
  부분의 텍스트를 추출합니다. 또한, 페이지 번호, HTML 태그 등 불필요한
  노이즈를 제거하는 정제 작업도 수행합니다.

In [ ]:
# 성분 섹션을 검출, 원문과 정제 형태로 반환
def slice_section_2_or_3(text: str,
                         prefer_numbers=(3,2),
                         max_span_lines=12000,
                         debug=False) -> Dict[str,str]:
    # 전체 텍스트를 줄 단위로 분리(개행 미포함) → 라인 리스트 생성
    lines = text.splitlines(keepends=False)

    # 루트 헤더 인덱스: 엄격 탐색 우선, 실패 시 완화 탐색 사용
    roots = collect_root_index_strict(lines) or collect_root_index_relaxed(lines)
    if not roots:
        if debug: print("[DBG] no root headers found")
        return {"raw":"", "clean":""}

    # 성분/조성 시작점 선정: 루트 후보 + 힌트 조합
    start_i = select_start_for_composition(lines, roots, prefer_numbers)
    if start_i is None:
        if debug: print("[DBG] no composition header found")
        return {"raw":"", "clean":""}

    # 다음 루트 경계: 상위 번호 증가 지점(3→4 등)
    end_i = find_next_root_after(lines, roots, start_i, min_span_lines=8)
    # 숫자 없는 제목형 등에서 실패 시, 제목 힌트 폴백
    if end_i is None:
        end_i = fallback_find_next_by_title(lines, start_i, scan_ahead=4000)
    # 그래도 실패하면 최대 슬라이스 길이 제한으로 강제 종료
    if end_i is None:
        end_i = min(len(lines), start_i + max_span_lines)

    # 원문 블록 추출: 시작~종료 범위 슬라이스
    raw_block = "\n".join(lines[start_i:end_i])

    # 정제: 페이지/마커/HTML 제거 + 연속 빈 줄 축약
    block, prev_blank = [], False
    for ln in raw_block.splitlines():
        # 1) 페이지 네비/마커/HTML/URL 제거 조건에 WEB_PAT 추가
        if (
            PAGE_NAV_PAT.search(ln)
            or PAGE_MARK_PAT.search(ln)
            or HTML_TAG_LINE_PAT.search(ln)
            or WEB_PAT.search(ln)
        ):
            continue
        t = ln.rstrip()
        if not t:
            if prev_blank:
                continue
            prev_blank = True
            block.append("")
        else:
            prev_blank = False
            block.append(ln)
    clean_block = "\n".join(block).strip()

    # 디버그 출력: 루트 수, 시작/종료 인덱스, 시작/종료 헤더 미리보기
    if debug:
        print(f"[DBG] roots_count={len(roots)} start_i={start_i} end_i={end_i}")
        print("[DBG] start header:", lines[start_i])
        if end_i < len(lines):
            print("[DBG] end header:", lines[end_i])

    # 원문과 정제 결과를 함께 반환
    return {"raw": raw_block, "clean": clean_block}

### msds_db_ingredients_LLM.py

[ingredients table 관련]

Ollama를 사용하여 MSDS 텍스트에서 성분 정보를 추출하는 모듈.

이 모듈은 대규모 언어 모델(LLM)을 활용하여 MSDS(물질안전보건자료) 문서의
구성성분 섹션 텍스트로부터 구조화된 성분 데이터를 JSON 형식으로 추출하는
기능을 제공합니다.

주요 기능:
- `FEW_SHOT` 프롬프트: 모델에게 역할, 스키마, 규칙, 그리고 다양한 예시
  (Few-shot examples)를 제공하여 정확한 결과물을 유도합니다.
- `build_prompt_for_sds`: 입력 텍스트와 사전 정의된 프롬프트를 결합하여
  LLM에 전달할 최종 프롬프트를 구성합니다.
- `ask_ollama`: 로컬 Ollama API에 HTTP POST 요청을 보내 모델의 응답을 받습니다.
- `extract_ingredients_with_ollama`: 전체 파이프라인을 실행하여 텍스트를
  입력받고, 최종적으로 파싱된 JSON 데이터를 반환합니다.

In [ ]:
MODEL = "qwen2.5:14b-instruct-q4_K_M"
OLLAMA_URL = "http://127.0.0.1:11434/api/generate"


FEW_SHOT = """
System:
You are a chemical composition extraction expert. Extract structured data strictly following the JSON schema. Return only a valid JSON array; no prose, no markdown, no code fences.


Schema (all keys required; use null when unknown):
[
  {
    "name": string,
    "synonym": string[],
    "cas": string|null,                  // keep original cell text (e.g., '영업 비밀', 'not disclosed', or a CAS number)
    "ec_number": string|null,
    "concentration": {
      "raw": string|null,                // keep the original cell text exactly (e.g., '영업 비밀', '>99% (w/w)')
      "value": number|null,
      "min": number|null,
      "max": number|null,
      "unit": string|null,               // normalized unit (e.g., '%', 'ppm', ...)
      "basis": string|null,              // 'w/w' or 'v/v' only when explicitly present
      "op_min": string|null,             // one of '>', '>=' or null
      "op_max": string|null              // one of '<', '<=' or null
    },
    "additional_info": {}
  }
]


Normalization rules:
- Copy original text literally into: cas (raw cell), concentration.raw (raw cell).
- Parse helpers:
  - For concentration, if raw contains numbers and explicit unit/op/basis, fill value/min/max/unit/basis/op_*; otherwise leave them null.
- Headers mapping (KR/EN): "Chemical name/화학 물질명"→name, "Synonyms/관용명"→synonym, "CAS-No./CAS번호"→cas, "Concentration/함유량(%)"→concentration.
- Percent unit: normalized to '%' if percentage.
- Basis mapping: 'wt%', '(w/w)', 'w/w' → basis='w/w'; 'vol%', '(v/v)', 'v/v' → basis='v/v'.
- Ranges: 'a–b%', 'a-b%', 'a to b%' → min=a, max=b, value/op_*=null.
- Operators: '>x%' → min/op_min='>'; '≥x%' → min/op_min='>='; '<x%' → max/op_max='<'; '≤x%' → max/op_max='<='.
- Approximate: '~x%' or 'about x%' → value=x only.
- Split synonyms by comma, semicolon, slash, vertical bar, or newline; trim/deduplicate; keep parentheses with their token.
- Missing pieces: always include keys; use null/[] accordingly.


Output constraints:
- Return a single JSON array only; no comments or fences.


Few-shot examples:


Example A:
Section:
| Chemical name | Synonyms | CAS-No. | Concentration |
| Hydrogen | HYDROGEN GAS | 1333-74-0 | >99% |
Expected JSON:
[
  {"name":"Hydrogen","synonym":["HYDROGEN GAS"],"cas":"1333-74-0","ec_number":null,
   "concentration":{"raw":">99%","value":null,"min":99,"max":null,"unit":"%","basis":null,"op_min":">","op_max":null},
   "additional_info":{}}
]


Example B:
Section:
| 화학 물질명 | 관용명 | CAS번호 | 함유량 (%) |
| 수소 | HYDROGEN GAS, HYDROGEN | 1333-74-0 | >99% |
Expected JSON:
[
  {"name":"Hydrogen","synonym":["HYDROGEN GAS","HYDROGEN"],"cas":"1333-74-0","ec_number":null,
   "concentration":{"raw":">99%","value":null,"min":99,"max":null,"unit":"%","basis":null,"op_min":">","op_max":null},
   "additional_info":{}}
]


Example C:
Section:
Propoxylated Sorbitol 50-60 % w/w; Propoxylated glycerol 40-50% (w/w)
Expected JSON:
[
  {"name":"Propoxylated Sorbitol","synonym":[],"cas":null,"ec_number":null,
   "concentration":{"raw":"50–60 % w/w","value":null,"min":50,"max":60,"unit":"%","basis":"w/w","op_min":null,"op_max":null},
   "additional_info":{}},
  {"name":"Propoxylated glycerol","synonym":[],"cas":null,"ec_number":null,
   "concentration":{"raw":"40–50% (w/w)","value":null,"min":40,"max":50,"unit":"%","basis":"w/w","op_min":null,"op_max":null},
   "additional_info":{}}
]


Example D:
Section:
Hydrogen; H2 | CAS 1333-74-0 | ≥99.999 vol% (v/v)
Acetone ~0.5 wt%
Expected JSON:
[
  {"name":"Hydrogen","synonym":["H2"],"cas":"1333-74-0","ec_number":null,
   "concentration":{"raw":"≥99.999 vol% (v/v)","value":null,"min":99.999,"max":null,"unit":"%","basis":"v/v","op_min":">=","op_max":null},
   "additional_info":{}},
  {"name":"Acetone","synonym":[],"cas":null,"ec_number":null,
   "concentration":{"raw":"~0.5 wt%","value":0.5,"min":null,"max":null,"unit":"%","basis":"w/w","op_min":null,"op_max":null},
   "additional_info":{}}
]


Example E (미기재/혼합물):
Section:
3. 구성성분의 명칭 및 함유량
이 제품의 물질은 혼합물로 구성
물질안전보건자료에 기재된 구성성분 외에 다른 구성성분은 산업안전보건법 상 유해인자 분류기준에 해당되지 않음
Expected JSON:
[
  {"name":"혼합물","synonym":["Mixture"],"cas":null,"ec_number":null,
   "concentration":{"raw":null,"value":null,"min":null,"max":null,"unit":null,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{"reason":"성분 미기재"}}
]


Example F (영업 비밀 보존):
Section:
| Chemical name | Synonyms | CAS-No.     | Concentration   |
| Component A   | -        | 영업 비밀   | 영업 비밀       |
Expected JSON:
[
  {"name":"Component A","synonym":[],"cas":"영업 비밀","ec_number":null,
   "concentration":{"raw":"영업 비밀","value":null,"min":null,"max":null,"unit":null,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{}}
]

Example G (영업 비밀 보존):
Section:
| Chemical name | Synonyms | CAS-No.     | Concentration   |
| 영업 비밀   | 영업 비밀        | 영업 비밀   | 영업 비밀       |
Expected JSON:
[
  {"name":"Component A","synonym":["영업 비밀"],"cas":"영업 비밀","ec_number":null,
   "concentration":{"raw":"영업 비밀","value":null,"min":null,"max":null,"unit":null,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{}}
]

Example G (영업 비밀 보존):
Section:
| Chemical name | Synonyms | CAS-No.     | Concentration   |
| Trade Secret   | Trade Secret        | Trade Secret   | Trade Secret       |
Expected JSON:
[
  {"name":"Trade Secret","synonym":["Trade Secret"],"cas":"Trade Secret","ec_number":null,
   "concentration":{"raw":"Trade Secret","value":null,"min":null,"max":null,"unit":null,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{}}
]

Example H (순서):
Section:

시클로헥사논

화학물질명

Cyclohexanone

관용명 및 이명(異名)

108-94-1

CAS번호 또는 식별번호

100%

함유량(%)
Expected JSON:
[
  {"name":"시클로헥사논","synonym":["Cyclohexanone"],"cas":"108-94-1","ec_number":null,
   "concentration":{"raw":"100%","value":100,"min":null,"max":null,"unit":%,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{}}
]

Example H (순서):
Section:

시클로헥사논

화학물질 A

화학물질명

Cyclohexanone

Component A

관용명 및 이명(異名)

108-94-1

123-12-1

CAS번호 또는 식별번호

100%

0-5%

함유량(%)
Expected JSON:
[
  {"name":"시클로헥사논","synonym":["Cyclohexanone"],"cas":"108-94-1","ec_number":null,
   "concentration":{"raw":"100%","value":100,"min":null,"max":null,"unit":%,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{}},
  {"name":"화학물질 A","synonym":["Component A"],"cas":"123-12-1","ec_number":null,
   "concentration":{"raw":"0-5%","value":100,"min":0,"max":5,"unit":%,"basis":null,"op_min":null,"op_max":null},
   "additional_info":{}}
]


Task:
Extract from the following input as "Section" and return only the JSON array.


Section:
{markdown_text}
"""

# 로컬 Ollama REST API에 POST 요청
def ask_ollama(prompt: str, model: str = MODEL, temperature: float = 0.0, timeout=180):
    try:
        r = requests.post(
            OLLAMA_URL,
            json={"model": model, "prompt": prompt, "stream": False, "options":{"temperature": temperature}},
            timeout=timeout
        )
        r.raise_for_status()
        return r.json().get("response","").strip()
    except requests.exceptions.RequestException as e:
        print(f"[ERROR] Ollama request failed: {e}")
        return ""

# FEW_SHOT(시스템 규칙/예제), 스키마/규칙/초안을 프롬프트로 구성
def build_prompt_for_sds(section_text: str, draft_items=None, max_chars=2800):
    schema = {
      "name": "string|null",
      "synonym": ["string"],
      "cas": "string|null",
      "ec_number": "string|null",
      "concentration": {
        "raw": "string|null",
        "value": "number|null",
        "min": "number|null",
        "max": "number|null",
        "unit": "string|null",
        "basis": "string|null",
        "op_min": "string|null",
        "op_max": "string|null"
      },
      "additional_info": {}
    }
    t = re.sub(r"\n{3,}", "\n\n", section_text).strip()[:max_chars]
    draft = draft_items or []
    return f"""
You are an SDS composition extractor. Output ONLY a JSON array following the schema below.


{FEW_SHOT}


Rules:
- Output MUST be a complete JSON array; missing values are null/empty.
- Copy original cell text literally into cas and concentration.raw.
- Parse-only fields:
  - concentration value/min/max/unit/basis/op_* only if explicit numeric/unit/operator present.
- Name priority: Korean as name, English as synonym when both exist.
- Split synonyms by comma, semicolon, slash, vertical bar, or newline; keep parentheses; deduplicate.


Schema:
{json.dumps(schema, ensure_ascii=False, indent=2)}


Section:
{t}


Draft items (may be empty):
{json.dumps(draft, ensure_ascii=False)}
"""

# build_prompt_for_sds로 프롬프트를 만들고 ask_ollama로 모델 응답을 수신
def extract_ingredients_with_ollama(sec_text: str, draft_items=None):
    prompt = build_prompt_for_sds(sec_text, draft_items=draft_items)
    txt = ask_ollama(prompt)
    m = re.search(r"\[\s*\{.*\}\s*\]", txt, re.S)
    if not m:
        m = re.match(r"\s*\[.*\]\s*$", txt, re.S)
    if not m:
        print("[ERROR] Failed to find a valid JSON array in LLM response.")
        return []
    try:
        return json.loads(m.group(0))
    except Exception as e:
        print(f"[ERROR] Failed to parse JSON from LLM response: {e}")
        return []


## msds_db_ingredients_postprocess.py

[ingredients table 관련]
LLM이 추출한 MSDS 성분 데이터 후처리(Post-processing) 모듈.

이 모듈은 대규모 언어 모델(LLM)이 추출한 초기 JSON 형식의 성분 데이터에 대해
정확성을 높이고 데이터 형식을 표준화하며, 누락된 정보를 보강하는 다양한
함수들을 제공합니다.

주요 기능:
- `normalize_unit_basis`: 농도의 단위와 기준('w/w', 'v/v')을 표준화합니다.
- `postprocess_synonyms`: 쉼표, 세미콜론 등으로 연결된 동의어 문자열을
  개별 토큰으로 분리하고 중복을 제거합니다.
- `enrich_cas_and_conc`: 구조화된 농도 값(min, max, value)을 기반으로
  원본 텍스트(`raw`) 필드를 재생성하는 등 데이터를 보강합니다.
- `fix_name_mixture`: '혼합물' 또는 'Mixture'와 같은 이름과 동의어를
  일관된 형식으로 표준화합니다.

In [ ]:
# 원문 텍스트(raw_text)에서 혼합 기준(basis: 'w/w' 또는 'v/v') 단서를 탐지
def infer_basis_from_text(raw_text: str) -> Optional[str]:
    t = raw_text or ""
    for rx, basis in BASIS_PATTERNS:
        if rx.search(t):
            return basis
    return None

# item['concentration']의 단위/기준을 표준화하고 누락 시 텍스트에서 basis를 추론
def normalize_unit_basis(item: Dict, raw_text: str) -> Dict:
    conc = item.get("concentration") or {}
    unit = (conc.get("unit") or "")
    basis = conc.get("basis")

    # 1) unit에 포함된 키워드로 basis/단위 보정
    u = unit.strip().lower()
    detected_basis = None
    if "w/w" in u or "wt%" in u or "중량" in u:
        detected_basis = "w/w"
        unit = "%"
    elif "v/v" in u or "부피" in u:
        detected_basis = "v/v"
        unit = "%"

    # 2) 화이트리스트 외 단위 표현을 흔한 변형 규칙으로 정규화
    if unit not in UNIT_WHITELIST and unit != "":
        if u in {"% w/w", "%(w/w)", "wt%", "중량 %"}:
            unit = "%"
            detected_basis = detected_basis or "w/w"
        elif u in {"% v/v", "%(v/v)", "부피 %"}:
            unit = "%"
            detected_basis = detected_basis or "v/v"
    
    # 3) basis 누락 시 원문에서 추론
    if not basis:
        basis_text = infer_basis_from_text(raw_text)
        if basis_text:
            basis = basis_text

    # 4) unit에서 감지된 basis가 우선
    if detected_basis:
        basis = detected_basis

    # 5) None 정규화 후 반영
    conc["unit"]  = unit if unit else None
    conc["basis"] = basis if basis else None
    item["concentration"] = conc
    return item

# 동의어 문자열 하나를 구분자(DELIMS_RX) 기준으로 토큰 분할
def split_synonyms_token(s: str) -> List[str]:
    s = (s or "").strip()
    if not s:
        return []
    parts = [p.strip() for p in re.split(DELIMS_RX, s) if p.strip()]
    return parts

# 각 항목의 synonym 필드를 구분자 분할 → 전개 → 중복 제거(순서 보존)로 정제
def postprocess_synonyms(items: List[Dict]) -> List[Dict]:
    out=[]
    for it in items:
        syns = it.get("synonym") or []
        new=[]
        for s in syns:
            new.extend(split_synonyms_token(s))
        seen=set(); syn_norm=[]
        for x in new:
            if x not in seen:
                seen.add(x); syn_norm.append(x)
        it["synonym"] = syn_norm
        out.append(it)
    return out

# 항목 리스트에 대해 cas/concentration 후처리를 보강
def enrich_cas_and_conc(items: List[Dict]) -> List[Dict]:
    out=[]
    for it in items:
        # raw가 비었지만 구조화 값이 있으면 raw 문자열을 조립
        conc = it.get("concentration") or {}
        if conc.get("raw") is None:
            parts=[]
            if conc.get("op_min") and conc.get("min") is not None:
                parts.append(f"{conc['op_min']}{conc['min']}")
            elif conc.get("min") is not None and conc.get("max") is not None:
                parts.append(f"{conc['min']}-{conc['max']}")
            elif conc.get("value") is not None:
                parts.append(str(conc["value"]))
            if conc.get("unit"): parts.append(conc["unit"])
            if conc.get("basis"): parts.append(f"({conc['basis']})")
            conc["raw"] = " ".join(parts) if parts else None
            it["concentration"] = conc

        out.append(it)
    return out

# '혼합물/Mixture' 이름/동의어 표기를 일관화
def fix_name_mixture(items: List[Dict]) -> List[Dict]:
    out=[]
    for it in items:
        name = (it.get("name") or "").strip()
        syns = [s.strip() for s in (it.get("synonym") or []) if s and s.strip()]
        
        # case 1: 이름이 영어 'mixture' → 한국어 '혼합물'로 표준화하고 동의어에 'Mixture' 선두 보장
        if EN_MIX_RX.match(name):
            syns = [s for s in syns if not EN_MIX_RX.match(s)]
            syns.insert(0, "Mixture")
            it["name"] = "혼합물"
            it["synonym"] = syns
        # case 2: 이름이 한국어 '혼합물' → 동의어에 영어 'Mixture' 보장
        elif KO_MIX_RX.match(name):
            if not any(EN_MIX_RX.match(s) for s in syns):
                syns.insert(0, "Mixture")
                it["synonym"] = syns
        # case 3: 이름이 둘 다 아니나 동의어에 '혼합물'이 포함 → 이름을 '혼합물'로 변경
        else:
            has_ko = any(KO_MIX_RX.match(s) for s in syns)
            if has_ko:
                it["name"] = "혼합물"
                syns = [s for s in syns if not KO_MIX_RX.match(s)]
                if not any(EN_MIX_RX.match(s) for s in syns):
                    syns.insert(0, "Mixture")
                it["synonym"] = syns
        out.append(it)
    return out



## msds_db_ingredients_pipeline.py

[ingredients table 관련]
MSDS 구성성분 추출을 위한 모듈.

이 모듈은 MSDS(물질안전보건자료)의 전체 텍스트로부터 구성성분 정보를 추출하는 함수를 제공합니다.

주요 기능:
- `extract_section23_and_ingredients`: 전체 텍스트를 입력받아,
  내부적으로 슬라이싱, LLM을 이용한 정보 추출, 후처리 단계를 모두
  수행하여 최종적으로 정제된 성분 데이터 리스트를 반환하는 핵심 함수입니다.

데이터 처리 흐름:
1.  **섹션 슬라이싱**: `msds_db_ingredients_slicer` 모듈을 사용하여
    텍스트에서 구성성분 정보가 담긴 부분(주로 섹션 2 또는 3)을 잘라냅니다.
2.  **LLM 정보 추출**: `msds_db_ingredients_LLM` 모듈을 통해 잘라낸 텍스트를
    Ollama LLM에 전달하고, 구조화된 JSON 형식의 성분 데이터를 추출합니다.
3.  **데이터 후처리**: `msds_db_ingredients_postprocess` 모듈의 함수들을
    사용하여 LLM이 추출한 데이터의 정확도를 높이고, 형식을 표준화하며,
    누락된 정보를 보강합니다.

이 모듈은 다른 여러 모듈의 기능을 오케스트레이션하여, 복잡한 텍스트 처리
과정을 단일 함수 호출로 단순화하는 역할을 합니다.


In [ ]:
def extract_section23_and_ingredients(text: str, debug: bool = True) -> Dict:
    """ Section 2/3 추출 및 성분 분석 파이프라인 """
    if debug:
        print("\n[INFO] Section 2/3 슬라이싱 시작...")
    
    sec = slice_section_2_or_3(text, prefer_numbers=(3, 2), debug=debug)
    section2_3_text = sec["clean"]
    
    if debug:
        print("\n[INFO] LLM 호출 시작...")
    
    ingredients = extract_ingredients_with_ollama(section2_3_text)
    
    if debug:
        print("[INFO] LLM 호출 완료")
        print("[INFO] 후처리 시작...")
    
    ingredients = postprocess_synonyms(ingredients)
    ingredients = fix_name_mixture(ingredients)
    ingredients = [normalize_unit_basis(it, section2_3_text) for it in ingredients]
    ingredients = enrich_cas_and_conc(ingredients)
    
    if debug:
        print("[INFO] 후처리 완료")
    
    return {
        "ingredients": ingredients,
        "section2_3_text": section2_3_text,
        "section_info": {
            "start_idx": sec.get("start_idx"),
            "end_idx": sec.get("end_idx"),
            "found_section": sec.get("found_section", "unknown")
        }
    }

# 실행
## msds_pipeline.py

[pipeline]

MSDS 자동 분석 및 데이터베이스 저장 파이프라인 실행 스크립트.

이 스크립트는 전체 MSDS(물질안전보건자료) 분석 파이프라인을 실행하는
메인 진입점(entry point)입니다.

프로세스 흐름:
1.  **데이터베이스 연결**: `get_mongo_postgre_db` 모듈을 사용하여 MongoDB와
    PostgreSQL에 연결합니다.
2.  **데이터 로딩**: MongoDB에서 분석할 특정 MSDS 문서를 `document_id`를
    이용하여 가져옵니다.
3.  **섹션 1 분석**: `msds_db_section1_pipeline` 모듈을 호출하여 문서에서
    섹션 1(제품 및 회사 정보)을 추출하고, 제품명과 회사명을 분석합니다.
4.  **섹션 2/3 분석**: `msds_db_ingredients_pipeline` 모듈을 호출하여
    문서에서 섹션 2 또는 3(구성성분 정보)을 추출하고, LLM을 통해 구조화된
    성분 데이터를 생성한 뒤 후처리합니다.
5.  **데이터베이스 스키마 초기화**: `msds_db_create_pg_tables` 모듈을 통해
    PostgreSQL에 데이터 저장을 위한 테이블과 인덱스가 준비되었는지 확인하고
    필요 시 생성합니다.
6.  **데이터 저장**: 분석이 완료된 제품 정보와 성분 정보를 PostgreSQL
    데이터베이스에 저장합니다.
7.  **결과 검증**: 데이터베이스에 저장된 내용의 일부를 다시 조회하여
    저장 과정이 성공적으로 완료되었는지 간단히 확인합니다.

실행 전 요구사항:
- `.env` 파일에 MongoDB 및 PostgreSQL 연결 정보가 올바르게 설정되어 있어야 합니다.
- 로컬 Ollama 서버가 실행 중이어야 합니다.
- 필요한 모든 파이썬 라이브러리가 설치되어 있어야 합니다.

In [ ]:
# ================= MongoDB에서 문서 가져오기 ==================
print("\n[INFO] MongoDB에서 MSDS 문서 조회 중...")

mongodb = get_mongodb()
if mongodb is None:
    raise RuntimeError("MongoDB 연결 실패")

collection = mongodb['msds_markdown_collection']
document_id = 'MOCK_MSDS_003'
doc = collection.find_one({'document_id': document_id})

if doc is None:
    raise RuntimeError("MongoDB에서 MSDS 문서를 찾을 수 없습니다.")

md_path = doc.get('file_name', 'unknown.pdf')
text = doc.get('content', '')

if not text:
    raise ValueError(f"문서 content가 비어있습니다. document_id={doc.get('document_id')}")

print(f"문서 로딩 완료: {doc.get('document_id')} - {md_path}")
print("="*80)

# ================= section 1 정보 추출 ==================
section1_result = extract_section1_and_fields_from_text(text)

# Section 1 추출 결과를 출력
print(json.dumps(section1_result, ensure_ascii=False, indent=2))

sec1_text, dbg = slice_section1_debug(text)
print("\n" + "="*80)
print("== Section 1 전체 슬라이스 ==")
print(f"(start_idx={dbg['start_idx']}, end_idx={dbg['end_idx']}, next={dbg['found_next']})")
print("-"*80)
print(sec1_text)
print("="*80 + "\n")


# ================= ingredients ==================
sec23_result = extract_section23_and_ingredients(text, debug=True)

# 디버그 출력용
print("\n== 최종 ingredients (JSON) ==")
print(json.dumps(sec23_result["ingredients"], ensure_ascii=False, indent=2))

print("\n" + "="*80)
print("== 추출된 Section 2/3 CLEAN 블록 ==")
print(f"(start_idx={sec23_result['section_info']['start_idx']}, "
      f"end_idx={sec23_result['section_info']['end_idx']}, "
      f"section={sec23_result['section_info']['found_section']})")
print("-"*80)
print(sec23_result["section2_3_text"])
print("="*80)

# ================= PostgreSQL 저장 ==================
# 1) PostgreSQL 연결
conn = get_postgres()

# 스키마 초기화
init_msds_schema(conn)

# 생성된 테이블 확인
check_tables_exist(conn)

# 2) 저장 호출 (파싱된 변수들을 그대로 전달)
try:
    pid = save_current_parse_to_postgres(
        conn=conn,
        md_path=md_path,
        section1_result=section1_result,
        sec1_text=sec1_text,
        ingredients=sec23_result["ingredients"],
        document_id=document_id
    )
    print("저장된 product_id =", pid)
except Exception as e:
    import traceback
    traceback.print_exc()
    raise

# 3) 검증(옵션)
with conn.cursor() as cur:
    cur.execute("SELECT id, file_name, product_name, company_name FROM products WHERE id = %s", (pid,))
    print("-"*80)
    print("products:", cur.fetchone())
    
    cur.execute("SELECT name, cas, conc_min, conc_max, conc_unit FROM ingredients WHERE product_id = %s ORDER BY id", (pid,))
    print("-"*80)
    print("ingredients:", cur.fetchall())

conn.close()
